## CREATE A VECTOR DB

1. Create a list of articles (link, date)

In [1]:
keywords = [
    # Energy Sector
    "XOM", "CVX", "COP", "EOG", "SLB",
    "Exxon", "Chevron", "ConocoPhillips", "EOG Resources Inc", "Schlumberger NV",
    "oil", "gas", "energy", "OPEC", "power",
    "electricity", "green", "utilities",

    # Financials Sector
    "JPM", "BAC", "WFC", "AXP", "MS",
    "JPMorgan", "Bank of America", "Wells Fargo", "American Express", "Morgan Stanley",
    "bank", "interest rates", "savings", "investment", "regulation",
    "inflation", "employment", "stock", "bond", "FED",
    "SEC", "NYSE", "NASDAQ", "S&P500",

    # Tech Sector
    "MSFT", "AAPL", "NVDA", "GOOGL", "META",
    "Microsoft", "Apple", "Nvidia", "Alphabet", "Meta",
    "AI", "Google", "cybersecurity", "fintech", "data", "cloud"
]

In [27]:
# ft articles data [heading, link, date] for a hardcoded keyword

import requests
from bs4 import BeautifulSoup
from datetime import datetime


news_data = []
keyword = 'aapl'
cutoff_date = datetime.strptime("01-Dec-2018", "%d-%b-%Y")

keyurl = f"https://www.ft.com/search?q={keyword}&page=1&sort=date&isFirstView=true"
headers = {
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36',
    'Accept': 'text/html,application/xhtml+xml,application/xml;q=0.9,image/webp,image/apng,*/*;q=0.8',
    'Accept-Language': 'en-US,en;q=0.9',
    'Accept-Encoding': 'gzip, deflate, br',
    'Connection': 'keep-alive',
    'Referer': 'https://www.ft.com/',
    'Origin': 'https://www.ft.com'
}

keyresult = requests.get(keyurl, headers=headers)
# keyresult = requests.get(keyurl)
print(f"HTTP Status Code: {keyresult.status_code}")

keyresult_content = keyresult.content
keysoup = BeautifulSoup(keyresult_content, "html.parser")
# print(keysoup.prettify())
maxpages = keysoup.find('span', {"class": "search-pagination__page"})
maxpages = int(maxpages.get_text(strip=True).split()[-1])
# maxpages += 5
print(maxpages)


for page in range(1, maxpages):
    url = f"https://www.ft.com/search?q={keyword}&page={page}&sort=date&isFirstView=true"
    result = requests.get(url, headers=headers)
    # print(page)
    # print(result.text)
    if "Sorry" in result.text: # maybe unnecessary
        print(f"Stopping iteration: Page {page} is not available.")
        break
    
    result_content = result.content
    soup = BeautifulSoup(result_content, "lxml")
    
    for article in soup.findAll("div", {"class": "o-teaser"}):
        heading = article.find("div", {"class": "o-teaser__heading"}).get_text(strip=True) #find(text=True), find text is outdated
        link = article.find("div", {"class": "o-teaser__heading"}).find('a', href=True)
        date = article.find('time', {"class": "o-teaser__timestamp-date"})
        
        # print(heading, link, date)
        if heading and link and date:
            article_date = datetime.strptime(date.text.strip(), "%B %d, %Y")
            if article_date > cutoff_date:
                news_data.append([heading, link['href'], date.text.strip()])

# print(pages)

print(f"Total articles collected in news_data: {len(news_data)}")
# print(news_data)

HTTP Status Code: 200
2
Total articles collected in news_data: 7
[['Is ChatGPT good at day trading? Probably not', '/content/0608b6a0-f586-430c-9e68-a177c544435a', 'March 28, 2024'], ['GraniteShares files to increase leverage on 17 ETFs', '/content/f26c4b3b-baa4-4586-9fa2-bf8d63fd457b', 'November 30, 2023'], ['Microsoft, and a macro lesson', '/content/d5ba4cab-7976-4352-b624-7654c6acf7c3', 'October 28, 2021'], ['High expectations leave no room for errorPremiumcontent', '/content/ed5c04dd-75cf-4338-b51c-869affebedde', 'July 7, 2020'], ['Markets not live, Thursday 30th January 2020', '/content/ad45f2d9-07e6-4b0c-a040-f831f8af5ae8', 'January 30, 2020'], ['China’s credit pulse has a kickPremiumcontent', '/content/3c35f688-36f6-11ea-a6d3-9a26f8c3cba4', 'January 14, 2020'], ['Investors Chronicle: Tesco, Serco, Apple', '/content/6fbc606c-1359-11e9-a581-4ff78404524e', 'January 11, 2019']]


In [2]:
#setup news_data, dont refresh

news_data = []
collected_links = set()

In [3]:
# setup keywords

# cutoff_date = datetime.strptime("15-Sep-2023", "%d-%b-%Y")

# keywords = [
#     # Energy Sector
#     "XOM", "CVX", "EOG", "SLB",
#     "Exxon", "Chevron", "ConocoPhillips", "Schlumberger",
#     "oil", "gas", "energy", "OPEC",
#     "electricity", "green", "utilities",

#     # Financials Sector
#     "JPM", "BAC", "WFC", "AXP",
#     "JPMorgan", "Bank of America", "Wells Fargo", "American Express", "Morgan Stanley",
#     "interest rates", "savings", "investment", "regulation",
#     "inflation", "employment", "FED", "SEC", "NYSE", "NASDAQ",

#     # Tech Sector
#     "MSFT", "AAPL", "NVDA", "GOOGL", "META",
#     "Microsoft", "Apple", "Nvidia", "Alphabet", "Meta",
#     "AI", "Google", "cybersecurity", "fintech", "data", "cloud"
# ]

# k1 = [
#     "XOM", "CVX", "EOG", "SLB" # energy 1  
# ]
# k2 = [
#     "Exxon", "Chevron", "ConocoPhillips", "Schlumberger" # energy 2
# ]
# k3 = [
#     "oil", "gas", "energy", "OPEC" # energy 3
# ]
# k4 = [
#     "electricity", "green", "utilities" # energy 4
# ]
import requests
from bs4 import BeautifulSoup
from datetime import datetime

headers = {
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36',
    'Accept': 'text/html,application/xhtml+xml,application/xml;q=0.9,image/webp,image/apng,*/*;q=0.8',
    'Accept-Language': 'en-US,en;q=0.9',
    'Accept-Encoding': 'gzip, deflate, br',
    'Connection': 'keep-alive',
    'Referer': 'https://www.ft.com/',
    'Origin': 'https://www.ft.com'
}

In [7]:
# ENERGY
# ft articles data [heading, link, date] for a list of keywords, need to split the keywords list bc it crashes

import csv
import os
import requests
from bs4 import BeautifulSoup
from datetime import datetime

headers = {
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36',
    'Accept': 'text/html,application/xhtml+xml,application/xml;q=0.9,image/webp,image/apng,*/*;q=0.8',
    'Accept-Language': 'en-US,en;q=0.9',
    'Accept-Encoding': 'gzip, deflate, br',
    'Connection': 'keep-alive',
    'Referer': 'https://www.ft.com/',
    'Origin': 'https://www.ft.com'
}

k1 = [
    "XOM", "CVX", "EOG", "SLB" # energy 1  
]
k2 = [
    "Exxon", "Chevron", "ConocoPhillips", "Schlumberger" # energy 2
]
k3 = [
    "oil", "gas", "energy", "OPEC" # energy 3
]
k4 = [
    "electricity", "green", "utilities" # energy 4
]
# collected_links = set()
file_name = 'news_data_energy.csv'

def write_to_csv(data, file_name):
    file_exists = os.path.isfile(file_name)
    with open(file_name, mode='a', newline='', encoding='utf-8') as file:
        writer = csv.writer(file)
        if not file_exists:
            # Write header only if the file does not exist
            writer.writerow(["Headline", "Link", "Date"])
        for row in data:
            writer.writerow(row)

def read_existing_links(file_name):
    if not os.path.isfile(file_name):
        return set()
    with open(file_name, mode='r', encoding='utf-8') as file:
        reader = csv.reader(file)
        next(reader, None)  # Skip the header
        existing_links = {row[1] for row in reader}
    return existing_links

existing_links = read_existing_links(file_name)
cutoff_date = datetime.strptime("15-Sep-2023", "%d-%b-%Y")


for keyword in k4:
    keyurl = f"https://www.ft.com/search?q={keyword}&page=1&sort=date&isFirstView=true"


    keyresult = requests.get(keyurl, headers=headers)

    keyresult_content = keyresult.content
    keysoup = BeautifulSoup(keyresult_content, "html.parser")
    try:
        maxpages = keysoup.find('span', {"class": "search-pagination__page"})
        maxpages = int(maxpages.get_text(strip=True).split()[-1])
    except (AttributeError, ValueError) as e:
        print(f"Skipping keyword '{keyword}' due to error: {e}")
        continue

    for page in range(1, maxpages):
        url = f"https://www.ft.com/search?q={keyword}&page={page}&sort=date&isFirstView=true"
        result = requests.get(url, headers=headers)
        
        if "Sorry" in result.text: # maybe unnecessary
            print(f"Stopping iteration: Page {page} is not available.")
            break
        
        result_content = result.content
        soup = BeautifulSoup(result_content, "lxml")
        
        skip_to_next_keyword = False
        page_data = []
        
        for article in soup.findAll("div", {"class": "o-teaser"}):
            heading = article.find("div", {"class": "o-teaser__heading"}).get_text(strip=True)
            link = article.find("div", {"class": "o-teaser__heading"}).find('a', href=True)
            date = article.find('time', {"class": "o-teaser__timestamp-date"})
            
            if heading and link and date:
                
                article_date = datetime.strptime(date.text.strip(), "%B %d, %Y")
                if article_date < cutoff_date:
                    skip_to_next_keyword = True
                    break
                if link not in existing_links:
                    page_data.append([heading, link, date.text.strip()])
                    existing_links.add(link)
        if page_data:
            write_to_csv(page_data, file_name)
        if skip_to_next_keyword:
            break
print('finished')

KeyboardInterrupt: 

In [11]:
# FINANCE
# ft articles data [heading, link, date] for a list of keywords, need to split the keywords list bc it crashes

import csv
import os
import requests
from bs4 import BeautifulSoup
from datetime import datetime

headers = {
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36',
    'Accept': 'text/html,application/xhtml+xml,application/xml;q=0.9,image/webp,image/apng,*/*;q=0.8',
    'Accept-Language': 'en-US,en;q=0.9',
    'Accept-Encoding': 'gzip, deflate, br',
    'Connection': 'keep-alive',
    'Referer': 'https://www.ft.com/',
    'Origin': 'https://www.ft.com'
}
# Financials Sector
#     "JPM", "BAC", "WFC", "AXP",
#     "JPMorgan", "Bank of America", "Wells Fargo", "American Express", "Morgan Stanley",
#     "interest rates", "savings", "investment", "regulation",
#     "inflation", "employment", "FED", "SEC", "NYSE", "NASDAQ",
k1 = [
    "JPM", "BAC", "WFC", "AXP" 
]
k2 = [
    "JPMorgan", "Bank of America", "Wells Fargo", "American Express", "Morgan Stanley"
]
k3 = [
    "interest rates", "savings", "investment", "regulation"
]
k4 = [
    "inflation", "employment", "FED", "SEC", "NYSE", "NASDAQ"
]
# collected_links = set()
file_name = 'news_data_fin.csv'

def write_to_csv(data, file_name):
    file_exists = os.path.isfile(file_name)
    with open(file_name, mode='a', newline='', encoding='utf-8') as file:
        writer = csv.writer(file)
        if not file_exists:
            # Write header only if the file does not exist
            writer.writerow(["Headline", "Link", "Date"])
        for row in data:
            writer.writerow(row)

def read_existing_links(file_name):
    if not os.path.isfile(file_name):
        return set()
    with open(file_name, mode='r', encoding='utf-8') as file:
        reader = csv.reader(file)
        next(reader, None)  # Skip the header
        existing_links = {row[1] for row in reader}
    return existing_links

existing_links = read_existing_links(file_name)
cutoff_date = datetime.strptime("15-Sep-2023", "%d-%b-%Y")


for keyword in k4:
    keyurl = f"https://www.ft.com/search?q={keyword}&page=1&sort=date&isFirstView=true"


    keyresult = requests.get(keyurl, headers=headers)

    keyresult_content = keyresult.content
    keysoup = BeautifulSoup(keyresult_content, "html.parser")
    try:
        maxpages = keysoup.find('span', {"class": "search-pagination__page"})
        maxpages = int(maxpages.get_text(strip=True).split()[-1])
    except (AttributeError, ValueError) as e:
        print(f"Skipping keyword '{keyword}' due to error: {e}")
        continue

    for page in range(1, maxpages):
        url = f"https://www.ft.com/search?q={keyword}&page={page}&sort=date&isFirstView=true"
        result = requests.get(url, headers=headers)
        
        if "Sorry" in result.text: # maybe unnecessary
            print(f"Stopping iteration: Page {page} is not available.")
            break
        
        result_content = result.content
        soup = BeautifulSoup(result_content, "lxml")
        
        skip_to_next_keyword = False
        page_data = []
        
        for article in soup.findAll("div", {"class": "o-teaser"}):
            heading = article.find("div", {"class": "o-teaser__heading"}).get_text(strip=True)
            link = article.find("div", {"class": "o-teaser__heading"}).find('a', href=True)
            date = article.find('time', {"class": "o-teaser__timestamp-date"})
            
            if heading and link and date:
                
                article_date = datetime.strptime(date.text.strip(), "%B %d, %Y")
                if article_date < cutoff_date:
                    skip_to_next_keyword = True
                    break
                if link not in existing_links:
                    page_data.append([heading, link, date.text.strip()])
                    existing_links.add(link)
        if page_data:
            write_to_csv(page_data, file_name)
        if skip_to_next_keyword:
            break
print('finished')

KeyboardInterrupt: 

In [16]:
# TECH
# ft articles data [heading, link, date] for a list of keywords, need to split the keywords list bc it crashes

import csv
import os
import requests
from bs4 import BeautifulSoup
from datetime import datetime

headers = {
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36',
    'Accept': 'text/html,application/xhtml+xml,application/xml;q=0.9,image/webp,image/apng,*/*;q=0.8',
    'Accept-Language': 'en-US,en;q=0.9',
    'Accept-Encoding': 'gzip, deflate, br',
    'Connection': 'keep-alive',
    'Referer': 'https://www.ft.com/',
    'Origin': 'https://www.ft.com'
}
#     # Tech Sector
#     "MSFT", "AAPL", "NVDA", "GOOGL", "META",
#     "Microsoft", "Apple", "Nvidia", "Alphabet", "Meta",
#     "AI", "Google", "cybersecurity", "fintech", "data", "cloud"
k1 = [
    "MSFT", "AAPL", "NVDA", "GOOGL", "META"
]
k2 = [
    "Microsoft", "Apple", "Nvidia", "Alphabet"
]
k3 = [
    "Meta", "AI"
]
k4 = [
    "cybersecurity", "fintech", "data", "cloud"
]

# collected_links = set()
file_name = 'news_data_tech.csv'

def write_to_csv(data, file_name):
    file_exists = os.path.isfile(file_name)
    with open(file_name, mode='a', newline='', encoding='utf-8') as file:
        writer = csv.writer(file)
        if not file_exists:
            # Write header only if the file does not exist
            writer.writerow(["Headline", "Link", "Date"])
        for row in data:
            writer.writerow(row)

def read_existing_links(file_name):
    if not os.path.isfile(file_name):
        return set()
    with open(file_name, mode='r', encoding='utf-8') as file:
        reader = csv.reader(file)
        next(reader, None)  # Skip the header
        existing_links = {row[1] for row in reader}
    return existing_links

existing_links = read_existing_links(file_name)
cutoff_date = datetime.strptime("15-Sep-2023", "%d-%b-%Y")


for keyword in k4:
    keyurl = f"https://www.ft.com/search?q={keyword}&page=1&sort=date&isFirstView=true"


    keyresult = requests.get(keyurl, headers=headers)

    keyresult_content = keyresult.content
    keysoup = BeautifulSoup(keyresult_content, "html.parser")
    try:
        maxpages = keysoup.find('span', {"class": "search-pagination__page"})
        maxpages = int(maxpages.get_text(strip=True).split()[-1])
    except (AttributeError, ValueError) as e:
        print(f"Skipping keyword '{keyword}' due to error: {e}")
        continue

    for page in range(1, maxpages):
        url = f"https://www.ft.com/search?q={keyword}&page={page}&sort=date&isFirstView=true"
        result = requests.get(url, headers=headers)
        
        if "Sorry" in result.text: # maybe unnecessary
            print(f"Stopping iteration: Page {page} is not available.")
            break
        
        result_content = result.content
        soup = BeautifulSoup(result_content, "lxml")
        
        skip_to_next_keyword = False
        page_data = []
        
        for article in soup.findAll("div", {"class": "o-teaser"}):
            heading = article.find("div", {"class": "o-teaser__heading"}).get_text(strip=True)
            link = article.find("div", {"class": "o-teaser__heading"}).find('a', href=True)
            date = article.find('time', {"class": "o-teaser__timestamp-date"})
            
            if heading and link and date:
                
                article_date = datetime.strptime(date.text.strip(), "%B %d, %Y")
                if article_date < cutoff_date:
                    skip_to_next_keyword = True
                    break
                if link not in existing_links:
                    page_data.append([heading, link, date.text.strip()])
                    existing_links.add(link)
        if page_data:
            write_to_csv(page_data, file_name)
        if skip_to_next_keyword:
            break
print('finished')

finished


In [30]:
print(len(news_data)) #110 rows after k1, 851
# k4 fully finished, energy 4745 rows 


625


In [17]:
import pandas as pd

# Read the CSV file
file_name = 'news_data.csv'
df = pd.read_csv(file_name)

# Group by the "Date" column and count the occurrences
date_counts = df['Date'].value_counts().reset_index()
date_counts.columns = ['Date', 'Count']

# Print the results
# print(date_counts)

# Sort the date_counts DataFrame by 'Count' in descending order
date_counts_sorted = date_counts.sort_values(by='Count', ascending=False)

# Get the last 365 rows
last_365_rows = date_counts_sorted.tail(365)
print(last_365_rows)


                 Date  Count
587      May 20, 2020      1
588      May 17, 2020      1
589      May 14, 2020      1
590      May 13, 2020      1
591      May 12, 2020      1
..                ...    ...
979  December 6, 2018      1
980  December 3, 2018      1
981    August 4, 2023      1
982     July 24, 2023      1
983      July 3, 2023      1

[365 rows x 2 columns]


In [18]:
file_name = 'news_data.csv'
df = pd.read_csv(file_name)

# Convert the 'Date' column to datetime format for processing
df['Date'] = pd.to_datetime(df['Date'], format='%B %d, %Y')

# Filter the DataFrame for dates in 2019
df_2019 = df[df['Date'].dt.year == 2019]

# Calculate the count of entries for 2019
count_2019 = df_2019.shape[0]

# Print the result
print(f"Total number of entries for 2019: {count_2019}")

Total number of entries for 2019: 205


In [19]:
# Read the CSV file
file_name = 'news_data.csv'
df = pd.read_csv(file_name)

# Convert the 'Date' column to datetime format
df['Date'] = pd.to_datetime(df['Date'], format='%B %d, %Y')

# Extract the year from the 'Date' column
df['Year'] = df['Date'].dt.year

# Filter the DataFrame for the years 2019 to 2024
filtered_df = df[df['Year'].between(2019, 2024)]

# Group by the 'Year' column and count the occurrences
year_counts = filtered_df['Year'].value_counts().sort_index()

# Print the results
print(year_counts)

Year
2019     205
2020     184
2021     223
2022     309
2023     558
2024    3255
Name: count, dtype: int64


In [20]:
# Read the CSV file
file_name = 'news_data.csv'
df = pd.read_csv(file_name)

# Convert the 'Date' column to datetime format
df['Date'] = pd.to_datetime(df['Date'], format='%B %d, %Y')

# Filter the DataFrame for the year 2024
df_2024 = df[df['Date'].dt.year == 2024]

# Group by the 'Date' column and count the occurrences
date_counts_2024 = df_2024['Date'].value_counts().reset_index()
date_counts_2024.columns = ['Date', 'Count']

# Sort the date_counts DataFrame by 'Count' in descending order
date_counts_2024_sorted = date_counts_2024.sort_values(by='Count', ascending=False)

# Get the last 365 rows
last_365_rows_2024 = date_counts_2024_sorted.tail(365)

# Print the results
print(last_365_rows_2024)

          Date  Count
0   2024-07-17     85
1   2024-07-19     72
2   2024-06-26     70
3   2024-07-18     64
4   2024-07-23     61
..         ...    ...
192 2024-01-04      1
193 2024-02-24      1
194 2024-01-10      1
195 2024-02-05      1
196 2024-03-02      1

[197 rows x 2 columns]


In [3]:
import pandas as pd

# Read the CSV file
file_name = 'news_data.csv'
df = pd.read_csv(file_name)

# Convert the 'Date' column to datetime format
df['Date'] = pd.to_datetime(df['Date'], format='%B %d, %Y')

# Filter the DataFrame for the months from October 2023 to July 2024
start_date = '2023-10-01'
end_date = '2024-06-30'
filtered_df = df[(df['Date'] >= start_date) & (df['Date'] <= end_date)]

# Group by year and month, then count the occurrences
filtered_df['YearMonth'] = filtered_df['Date'].dt.to_period('M')
month_counts = filtered_df['YearMonth'].value_counts().sort_index()

# Print the results
print(month_counts)

YearMonth
2023-10     68
2023-11     53
2023-12    105
2024-01    111
2024-02    181
2024-03    149
2024-04    264
2024-05    647
2024-06    748
Freq: M, Name: count, dtype: int64


/var/folders/w9/qwhj77nd0dn8pf5px6zk_xxw0000gn/T/ipykernel_40316/3684213754.py:16: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  filtered_df['YearMonth'] = filtered_df['Date'].dt.to_period('M')
